In [ ]:
import numpy as np
import scipy as sp
import os 
import sys
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import importlib

In [ ]:
util_path = 'C:/ASM/Dropbox/Developments/Jupyter/Eating/myutils' if 'C:' in os.getcwd() else './myutils'
sys.path.append(util_path)
import my_file_utils as mfileu
import my_steven_lab_utils as mslabu
import my_classification_utils as mcu
#importlib.reload(mdu)

In [ ]:
data = mfileu.read_file('data', 'lab_data_steven.pkl')
features = mfileu.read_file('features', 'lab_features_steven_right.pkl')

In [ ]:
def get_labels(indices, a, win_size):
    a = a[a[:, 3]!=2, :] # right hand only    
    wcount, acount = len(indices), len(a)  
    
    si = indices
    q1 = si + win_size//4
    mi = si + 2*win_size//4    
    q3 = si + 3*win_size//4    
    
    labels=np.zeros((wcount,))    
    for i in range(acount):         
        if a[i, 2]==1: #bite
            ix = a[i, 0]
            cond = (q1<=ix) & (ix<=q3)
            label = 1
        else:#sip
            ix1 = a[i, 0]
            ix2 = a[i, 1]
            cond = (mi>=ix1) & (mi<=ix2)
            label = 2
        
        labels[cond] = label
    return np.array(labels)

In [ ]:
def get_lopo_train_features_labels(data, features, exclude_subj):    
    train_x, train_y = [], []
    for subj in range(len(features)):        
        if subj==exclude_subj:
            continue
        for sess in range(len(features[subj])):             
            f = features[subj][sess][:, 1:]
            a = data[subj][sess]["annots"]            
            l = get_labels(features[subj][sess][:, 0], a, 5*16)            
            
            if len(train_x)==0:
                train_x = f
                train_y = l
            else:                
                train_x = np.concatenate((train_x, f))
                train_y = np.concatenate((train_y, l))
            
    return train_x, train_y
    

In [ ]:
for subj in [0, 1, 2, 3, 4, 5, 6, 100]:    
    train_x, train_y = get_lopo_train_features_labels(data=data, features=features, exclude_subj=subj)
    print("Subj:", subj, ", shapes:", train_x.shape, train_y.shape, "Neg, Bite, sip, other:", np.sum(train_y==0), np.sum(train_y==1), np.sum(train_y==2), np.sum(train_y==3))
    train_y[train_y==2] = 0
    print("Subj:", subj, ", shapes:", train_x.shape, train_y.shape, "Neg, Bite, sip, other:", np.sum(train_y==0), np.sum(train_y==1), np.sum(train_y==2), np.sum(train_y==3))
    
    clf = RandomForestClassifier(n_estimators =100, random_state=0, n_jobs=-1, verbose=1)
    #train_y = mcu.get_one_hot(train_y.astype(int), 3)
    clf.fit(train_x, train_y)    
    print("Training done!")

    mfileu.write_file('baseline_models', 'lab_'+str(subj)+'.pkl', clf)
    print("Model saved")
